## define `Robot structure` Class

In [1]:
import numpy as np 
import rospy 
import os 
import sys
sys.path.append("..")
from kinematics.class_structure import CHAIN

print("Done.")

Done.


In [2]:
import numpy as np 
import sys
sys.path.append("..")
from kinematics.utils.util_fk import *
from kinematics.utils.util_ik import * 
from kinematics.utils.util_structure import * 
from kinematics.utils.util_parser import PARSER 
from kinematics.utils.config import CONFIG 

class CAPSULE():
    def __init__(self, 
                 scale=[], 
                 cap_radius = float(0), 
                 cap_height = float(0), 
                 cap_p      = column_v(0,0,0), 
                 cap_R      = rot_e(), 
                 cap_center_p = column_v(0, 0, 0), 
                 cap_size     = column_v(0,0,0), 
                 cap_t_offset =np.eye(4)):

        self.scale    = scale 
        self.radius   = cap_radius
        self.height   = cap_height
        self.p        = cap_p
        self.R        = cap_R
        self.center_p = cap_center_p
        self.size     = cap_size
        self.T_offset = cap_t_offset
    

class JOINT():
    def __init__(self, 
                 name   = str('joint'), 
                 id     = int(0), 
                 mother = 0, 
                 child  =[0], 
                 q      = float(0), 
                 a = column_v(1,1,1), 
                 b = column_v(0,0,0),
                 p = column_v(0,0,0), 
                 R = rot_e(), 
                 R_offset = Rotation_E(), 
                 dq = float(0), 
                 type =''):

        self.name     = name 
        self.id       = id 
        self.mother   = mother
        self.child    = child 
        self.q        = q
        self.a        = a 
        self.b        = b 
        self.p        = p 
        self.R        = R 
        self.R_offset = R_offset
        self.dq       = dq 
        self.type     = type


class LINK(CAPSULE):
    def __init__(self, 
                 name       = str('link'), 
                 joint_id   = int(0), 
                 joint_name = str('mother_joint'), 
                 P_offset     = column_v(0,0,0), 
                 R_offset     = rot_e(), 
                 cap_center_p = column_v(0, 0, 0),
                 scale= [] , 
                 cap_radius   = float(0), 
                 cap_height   = float(0), 
                 cap_t_offset = np.eye(4),
                 mesh_path    = str('path'), 
                 cap_p = column_v(0,0,0), 
                 cap_R = rot_e(), 
                 color = "0.2 0.2 0.2 1", 
                 collision_type = "mesh", 
                 cap_size       = column_v(0,0,0)):
        CAPSULE.__init__(self, scale, cap_radius, cap_height, cap_p, cap_R, cap_center_p, cap_size, cap_t_offset)

        self.name           = name 
        self.joint_id       = joint_id 
        self.joint_name     = joint_name 
        self.P_offset       = P_offset
        self.R_offset       = R_offset 
        self.mesh_path      = mesh_path 
        self.color          = color 
        self.collision_type = collision_type

        # Capsule 
        self.cap = CAPSULE()
        self.cap.scale  = scale
        self.cap.radius = cap_radius 
        self.cap.height = cap_height
        self.cap.p      = cap_p
        self.cap.R      = cap_R
        self.cap.size   = cap_size #In case of box type
        self.cap.T_offset = cap_t_offset #In case of mesh type
        self.cap.center_p = cap_center_p #In case of mesh type

class CHAIN:
    def __init__(self, file_name="ir_gazebo/script/structure/utils/ur5e_onrobot.urdf",
                       base_offset=[0.1, 0, 0.5], 
                       verbose=False):
        self.parser = PARSER(file_name)
        self.joint  = []
        self.link   = [] 
        self.base_offset = base_offset
        self.verbose = verbose

    def fk_chain(self, idx_to):
        if idx_to == 0:
            joint_to = self.joint[idx_to]
            link_to  = self.link[idx_to]
            p = joint_to.b
            r = joint_to.R
            T_capsule = np.matmul(pr2t(p, r) ,link_to.T_offset)
            link_to.cap.center_p = t2p(T_capsule)
            return None 
        idx_fr = self.joint[idx_to-1].mother 
        if idx_fr != 0:
            link_fr  = self.link[idx_fr-1]
            link_to  = self.link[idx_to-1]
            joint_fr = self.joint[idx_fr-1]
            joint_to = self.joint[idx_to-1]
            joint_to.p = (np.matmul(joint_fr.R, joint_to.b) + joint_fr.p).astype(float)
            link_to.cap.p = joint_to.p + link_fr.P_offset 

            if (joint_to.a == column_v(1,1,1)).all():
                joint_to.R = np.matmul(joint_fr.R, joint_to.R_offset)
            else:   
                joint_fr_after = np.matmul(joint_fr.R, joint_to.R_offset)
                joint_to.R = np.matmul(joint_fr_after, rodrigues(joint_to.a, joint_to.q)).astype(float)

            T_capsule = np.matmul(pr2t(joint_to.p, joint_to.R) ,link_to.T_offset)
            link_to.cap.center_p = t2p(T_capsule)
            link_to.cap.R = t2r(T_capsule)

        for child_idx in self.joint[idx_to-1].child:
            self.fk_chain(child_idx)

    # Inverse Kinematics LM
    def aug_inverse_kinmatics_LM(self, variables, wn_pos, wn_ang):
        ctrl_num = variables["joint_ctrl_num"]
        # Get revolute_joint for IK 
        rev_joi    = get_rev_joi_chain(self.joint, ctrl_num)
        J, err, we = get_aug_ik_ingredients(self.joint, variables, wn_pos, wn_ang)
        self.fk_chain(1)
        Ek = np.matmul(err.T, we) 
        Ek = np.matmul(Ek, err) 
        while True: 
            dq, _ = damped_ls(J, err)
            move_joints(self.joint, rev_joi, dq)
            self.fk_chain(1)
            J, err, _ = get_aug_ik_ingredients(self.joint, variables) 
            Ek2 = np.matmul(err.T, we)
            Ek2 = np.matmul(Ek2, err)
            if Ek2 < 1e-12:
                break
            elif Ek2 < Ek:
                Ek = Ek2 
            else: 
                move_joints(self.joint, rev_joi, -dq) #revert
                self.fk_chain(1)
                break 

    # IK Solver
    def get_q_from_ik(self, variable, _wn_pos=1/0.3, _wn_ang=1/(2*np.math.pi)):
        self.fk_chain(1)
        self.aug_inverse_kinmatics_LM(variable, _wn_pos, _wn_ang)
        q_list = get_q_chain(self.joint)
        return q_list 

    # Add joint to robot 
    def add_joi_to_robot(self):
        # Other joints      
        for idx, joint in enumerate(self.parser.joint): 
            if idx == 0: 
                self.joint.append(JOINT(name=joint['name'], 
                                            id=joint['id'], 
                                            mother = self.parser.get_mother(joint['parent']), 
                                            child=self.parser.get_child_joint_tree(joint["child"]), q=0, 
                                            a=column(joint['axis']), 
                                            b=column(joint['xyz']), 
                                            p=column_v(self.base_offset[0], 
                                                       self.base_offset[1],
                                                       self.base_offset[2]), # Attached position offset from the world 
                                            R=np.eye(3), 
                                            R_offset=make_rotation(rad=joint['rpy']),
                                            type=joint['type']))          
            
            else: 
                self.joint.append(JOINT(name=joint['name'], 
                                            id=joint['id'], 
                                            mother = self.parser.get_mother(joint['parent']), 
                                            child=self.parser.get_child_joint_tree(joint["child"]), q=0, 
                                            a=column(joint['axis']), 
                                            b=column(joint['xyz']), 
                                            p=column_v(0, 0, 0), 
                                            R=np.eye(3), 
                                            R_offset=make_rotation(rad=joint['rpy']),
                                            type=joint['type']))   
                                             

        """ Verbose Function """
        if self.verbose:
            for joi in self.joint: 
                print("Name: {}, ID: {}, Mother_joint: {}, Child_joint: {}, Type: {}, Axis: {}, B_offset:{}".
                    format(joi.name.ljust(35), str(joi.id).ljust(2), str(joi.mother).ljust(2), 
                           str(joi.child).ljust(11), joi.type.ljust(10), str(joi.a.T).ljust(16), 
                           str(joi.b.T).ljust(5)))
                print("Current Position: {}\nCurrent Rotation: \n{}\n".format(joi.p.T, joi.R))         

    def add_link_to_robot(self):
        """ CONNECT LINK TREE """
        for link, capsule in zip(self.parser.link, CONFIG["capsule"]):
            self.link.append(LINK(name=link['name'], 
                                        joint_id   = self.parser.get_link_joint_id(link['name']), 
                                        joint_name = self.parser.get_link_joint_name(link['name']), 
                                        P_offset   = column(link["P_offset"]), 
                                        R_offset   = make_rotation(rad=link["R_offset"]), 
                                        scale      = link['scale'], 
                                        mesh_path  = link["filepath"], 
                                        collision_type = link["collision_type"], 
                                        cap_p        = column_v(0.0, 0.0, 0.0), 
                                        cap_R        = np.eye(3),
                                        cap_radius   = capsule["radius"],
                                        cap_size     = capsule["size"],
                                        cap_t_offset = capsule["T_offset"],
                                        cap_center_p = column_v(0.0, 0.0, 0.0),
                                        cap_height   = capsule["height"],
                                        color        = get_color_from_urdf(link['color'])))
        self.fk_chain(1)
        # CONNECT ALL JOINT/LINKS TO THE ROBOT 
        if self.verbose:
            print('len_link', len(self.link))
            for link in self.link:
                print("Name: {}, Connected Joint ID: {}, Connected Joint name: {}, \nMesh Path: {}, \
                    \nCollision Type: {}, Radius: {}, Heihgt: {}, Cap_Position: {}, \nCenter Position: {}\n".
                    format(link.name.ljust(30), str(link.joint_id).ljust(2), link.joint_name.ljust(30), 
                           link.mesh_path, link.collision_type, str(link.radius).ljust(10), str(link.height).ljust(10), 
                           link.cap.p.T, link.cap.center_p))



### Load pre-defined `UR5e` robot `urdf`

In [24]:
file_name = "../../urdf/ur5e_onrobot.urdf"
chain = CHAIN(file_name=file_name, base_offset=[0.18,0,0.79], verbose=False)
chain.add_joi_to_robot()
chain.add_joi_to_robot()
chain.add_link_to_robot()

# chain.fk_chain(1)


## Set `UR5` Robot

In [25]:
import sys 
sys.path.append("..")
import cv2 
from get_rostopic import RealsenseD435i
import numpy as np 
import rospy 
import matplotlib.pyplot as plt 
from utils.utils_pcl import *
from utils.utils_pointcloud import * 
print("Done.")


Done.


In [26]:
import rospy 
import time 
import numpy as np 
import message_filters
from sensor_msgs.msg import JointState


### define `UR` Class

In [27]:
class UR_Test():
    def __init__(self):
        self.tick = 0
        self.joint_list = None 
        self.joint_sub = rospy.Subscriber('/joint_states', JointState, self.joint_callback) 

        tic_temp=0 

        while self.tick<2: 
            time.sleep(1e-3)
            tic_temp=tic_temp+1

            if tic_temp>5000: 
                print ("[ERROR] GET JOINTS")
                break 
            
    def joint_callback(self, joint_msg):
        """
            Get joint values about each joint.
        """
        self.tick+=1 
        self.joint_list = joint_msg 

    # def fk_chain(self, idx_to):
    #     # self.joint_list = 
        
    #     if idx_to == 0:
    #         pass
    #     elif idx_to != 0:
    #         link_mother =  self.joint

    #     return 0

#### before run below code, you should run this line `roslaunch ur_modern_driver ur5_bringup.launch robot_ip:=192.168.0.100`  in terminal.

In [7]:
!roslaunch ur_modern_driver ur5_bringup.launch robot_ip:=192.168.0.100

... logging to /home/rilab/.ros/log/729ee8d4-a85d-11ed-b717-c7cd5ec2be1a/roslaunch-rilab-XPS-15-9500-55372.log
Checking log directory for disk usage. This may take a while.
Press Ctrl-C to interrupt
Done checking log file disk usage. Usage is <1GB.
]2;/home/rilab/catkin_qd/src/ur_modern_driver/launch/ur5_bringup.launch
xacro: in-order processing became default in ROS Melodic. You can drop the option.
started roslaunch server http://rilab-XPS-15-9500:40129/

SUMMARY

PARAMETERS
 * /robot_description: <?xml version="1....
 * /rosdistro: noetic
 * /rosversion: 1.15.15
 * /ur_driver/base_frame: base
 * /ur_driver/max_payload: 5.0
 * /ur_driver/max_velocity: 10.0
 * /ur_driver/min_payload: 0.0
 * /ur_driver/prefix: 
 * /ur_driver/robot_ip_address: 192.168.0.100
 * /ur_driver/servoj_time: 0.008
 * /ur_driver/tool_frame: tool0_controller

NODES
  /
    robot_state_publisher (robot_state_publisher/robot_state_publisher)
    ur_driver (ur_modern_driver/ur_driver)

auto-starting new master
pro

In [29]:
p = chain.joint[2].p
R = chain.joint[2].R

In [5]:
rospy.init_node('test')
ur_test = UR_Test()

print(ur_test.joint_list.position)
joint_value = ur_test.joint_list.position

(-1.5739653746234339, -2.314601560632223, 2.145999256764547, 1.740140123958252, 0.7839429378509521, -1.5683963934527796)


In [ ]:
for i in range(len(joint_value)):
    chain.joint[i].q = joint_value[i]
    print(chain.joint[i].q)

# forward kinematics: correspond to each joint values.
chain.fk_chain(1)


In [ ]:
cv2.hconcat((R, p))

In [31]:
T_ = cv2.hconcat((R, p))
T_

array([[ 4.89658886e-12,  0.00000000e+00,  1.00000000e+00,
         1.80000000e-01],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         1.38000000e-01],
       [-1.00000000e+00,  0.00000000e+00,  4.89658886e-12,
         9.53000000e-01]])

In [ ]:
R_cam_joint = chain.joint[len(joint_value)-1].R
p_cam_joint = chain.joint[len(joint_value-1)].p

T_cam_joint = cv2.hconcat((R_cam_joint, p_cam_joint))